In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import cv2
from scipy.signal import filtfilt, butter, find_peaks

In [2]:
def rotateImg(img, angle):
    center = tuple(np.array(img.shape[1::-1]) / 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Apply the rotation matrix to the input image
    rotated = cv2.warpAffine(img, M, img.shape[1::-1], flags=cv2.INTER_LINEAR)
    return rotated


In [3]:
def plotTwoCurves(x, y1, y2):
    fig, ax = plt.subplots()
    ax.plot(x, y1, color='red', label='top')

    # Plot the second curve (in red) on the same axis
    ax.plot(x, y2, color='blue', label='bottom')

    # Set the axis labels and title
    ax.set_xlabel('x (pixel pos''n)')
    ax.set_ylabel('intensity')
    ax.set_title('top & bottom image intensity')
    # Add a legend to the plot
    ax.legend()
    # Display the plot
    plt.show()

In [34]:
raw_data_folder = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint'    # folder with image inputs
processed_folder = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed'  

In [35]:
#fiter_coeffs
f_b, f_a = butter(2, 1/10, btype='lowpass', analog=False)

In [36]:
if os.path.isdir(processed_folder) is False:
    os.makedirs(processed_folder)
    for folder in glob.glob(os.path.join(raw_data_folder, '*')):
        if os.path.isdir(folder):
            delimiter = '\\'
            substrs = folder.split(delimiter)
            folderName = substrs[-1]
            #if folderName != '044':     #in case you want to work on a specific class
            #    continue
            for file in glob.glob(os.path.join(folder, '*')):
                if os.path.isfile(file) and "_F_" in os.path.basename(file):  #only process top view images
                    print('Processing file '+file)
                    delimiter = '\\'
                    substrs = file.split(delimiter)
                    fileName = substrs[-1]
                    img = cv2.imread(file)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.medianBlur(img, 5)  # median filter image
                    [ln, wd, _] = img.shape
                    #make sure the images all have the same alignment
                    if wd > ln:
                        img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
                        [ln, wd, _] = img.shape
                    gr_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
                    rd_img = img[:,:,1]

                    # check the intensity of top and bottom parts of the image
                    topStrip = rd_img[1:ln//55, :]
                    topStrip = np.mean(topStrip, axis=0)/255
                    bottomStrip = rd_img[54*ln//55:, :]
                    bottomStrip = np.mean(bottomStrip, axis=0)/255

                    topStrip = filtfilt(f_b, f_a, topStrip)
                    bottomStrip = filtfilt(f_b, f_a, bottomStrip)

                    # count the number of peaks. Fingers generate multiple peaks and wrist generates a single peak
                    minTop = np.min(topStrip)
                    maxTop = np.max(topStrip)
                    minBottom = np.min(bottomStrip)
                    maxBottom = np.max(bottomStrip)
                    diffTop = maxTop-minTop
                    diffBottom = maxBottom-minBottom
                    avgDiff = diffTop/2 + diffBottom/2
                    topStrip = np.pad(topStrip, (1, 1), mode='constant', constant_values=minTop)
                    bottomStrip = np.pad(bottomStrip, (1, 1), mode='constant', constant_values=minBottom)

                    count = 0
                    peaksTop = np.array([0])
                    peaksBottom = np.array([0])

                    #plotTwoCurves(range(wd+2), topStrip-np.mean(topStrip), bottomStrip-np.mean(bottomStrip))

                    # make sure the fingers are on top. If not rotate image
                    while count < 3 and  len(peaksTop) == len(peaksBottom):
                        peaksTop, _ = find_peaks(topStrip, prominence=avgDiff/(3+count), distance=wd//9, width=wd//20)
                        peaksBottom, _ = find_peaks(bottomStrip, prominence=avgDiff / (3+count), distance=wd//9, width=wd//20)
                        if len(peaksTop) < len(peaksBottom):
                            img = cv2.rotate(img, cv2.ROTATE_180)
                            break
                        count += 1

                    if len(peaksTop) == len(peaksBottom):
                        darkBottom = bottomStrip < diffBottom*0.1 + minBottom
                        darkTop = topStrip < diffTop*0.1 + minTop
                        if np.count_nonzero(darkBottom) < np.count_nonzero(darkTop):
                            img = cv2.rotate(img, cv2.ROTATE_180)

                    # save processed image
                    savePath = processed_folder + '/' + folderName + '/'
                    if os.path.isdir(savePath) is False:
                        os.makedirs(savePath)
                    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                    cv2.imwrite(savePath + fileName, img)

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\001\001_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\001\001_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\001\001_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\001\001_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\001\001_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\001\001_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\001\001_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\001\001_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\001\001_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\001\001_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\002\002_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\002\002_F_1.JPG
Processing file C:/Users/Vineet Roy/Down

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\011\011_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\011\011_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\011\011_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\011\011_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\011\011_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\011\011_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\011\011_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\012\012_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\012\012_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\012\012_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\012\012_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\012\012_F_4.JPG
Processing file C:/Users/Vineet Roy/Down

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\021\021_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\021\021_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\021\021_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\021\021_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\022\022_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\022\022_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\022\022_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\022\022_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\022\022_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\022\022_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\022\022_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\022\022_F_7.JPG
Processing file C:/Users/Vineet Roy/Down

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\031\031_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\032\032_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\032\032_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\032\032_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\032\032_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\032\032_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\032\032_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\032\032_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\032\032_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\032\032_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\032\032_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\033\033_F_0.JPG
Processing file C:/Users/Vineet Roy/Down

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\042\042_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\042\042_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\042\042_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\042\042_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\042\042_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\042\042_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\042\042_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\042\042_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\043\043_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\043\043_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\043\043_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\043\043_F_3.JPG
Processing file C:/Users/Vineet Roy/Down

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\052\052_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\052\052_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\052\052_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\052\052_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\052\052_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\053\053_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\053\053_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\053\053_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\053\053_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\053\053_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\053\053_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\053\053_F_6.JPG
Processing file C:/Users/Vineet Roy/Down

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\063\063_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\063\063_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\064\064_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\064\064_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\064\064_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\064\064_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\064\064_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\064\064_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\064\064_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\064\064_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\064\064_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\064\064_F_9.JPG
Processing file C:/Users/Vineet Roy/Down

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\074\074_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\074\074_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\074\074_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\074\074_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\074\074_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\074\074_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\074\074_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\074\074_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\074\074_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\075\075_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\075\075_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\075\075_F_2.JPG
Processing file C:/Users/Vineet Roy/Down

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\084\084_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\084\084_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\084\084_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\084\084_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\084\084_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\084\084_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\085\085_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\085\085_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\085\085_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\085\085_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\085\085_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint\085\085_F_5.JPG
Processing file C:/Users/Vineet Roy/Down

In [37]:
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [38]:
def plotImages(images_arr, numImages):
    fig, axes = plt.subplots(1,numImages, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [39]:
def resizeImage(img):
    output_ln = 408
    output_wd = 306
    [ln, wd, _] = img.shape
    if ln / wd > 1.3 and ln / wd < 1.4:
        img = cv2.resize(img, (output_wd, output_ln), interpolation=cv2.INTER_LINEAR)
    elif ln / wd > 1.7 and ln / wd < 1.82:
        bottom_crop = 0.05
        top_crop = 0.2
        top_rows = int(top_crop * ln)
        bottom_rows = int(bottom_crop * ln)
        cropped_img = img[top_rows:ln - bottom_rows, :]
        img = cv2.resize(cropped_img, (output_wd, output_ln), interpolation=cv2.INTER_LINEAR)
    return img

In [40]:
  # folder with image inputs
augmented_folder = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/augmented' 
processed_folder = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed' 

In [41]:
gen = ImageDataGenerator(rotation_range=5, width_shift_range=0.01, height_shift_range=0.01, shear_range=0.12, zoom_range=0.06, channel_shift_range=10, horizontal_flip=False)
numExtraImages = 3

In [42]:
if os.path.isdir(augmented_folder) is False:
    os.makedirs(augmented_folder, exist_ok=True)
    for folder in glob.glob(os.path.join(processed_folder, '*')):
        if os.path.isdir(folder):
            delimiter = '\\'
            substrs = folder.split(delimiter)
            folderName = substrs[-1]
            for file in glob.glob(os.path.join(folder, '*')):
                if os.path.isfile(file):
                    print('Processing file ' + file)
                    delimiter = '\\'
                 ll   substrs = file.split(delimiter)
                    fileName = substrs[-1]
                    assert os.path.isfile(file)

                    img = cv2.imread(file)
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = resizeImage(img)
                    image = np.expand_dims(img, 0)
                    # plot original image
                    #plt.imshow(image[0])

                    aug_iter = gen.flow(image)
                    aug_images = [next(aug_iter)[0].astype(np.uint8) for i in range(numExtraImages)]
                    # plot augmented images
                    #plotImages(aug_images, numExtraImages)

                    fileModified = fileName[:6] + '0' + fileName[6:]
                    if os.path.isdir(augmented_folder+'/'+folderName) is False:
                        os.makedirs(augmented_folder+'/'+folderName)

                    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                    cv2.imwrite(augmented_folder + '/' + folderName + '/' + fileModified, img)

                    for k in range(1, np.min([numExtraImages+1, 10])):
                        fileModified = fileName[:6]+chr(k+48)+fileName[6:]
                        img = cv2.cvtColor(aug_images[k-1], cv2.COLOR_RGB2BGR)
                        cv2.imwrite(augmented_folder+'/'+folderName+'/'+fileModified, img)

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\001\001_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\001\001_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\001\001_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\001\001_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\001\001_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\001\001_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\001\001_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\001\001_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\001\001_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\001\001_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\002\002_F_0.JPG
Processing

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\010\010_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\010\010_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\010\010_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\010\010_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\010\010_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\010\010_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\010\010_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\010\010_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\011\011_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\011\011_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\011\011_F_2.JPG
Processing

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\019\019_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\019\019_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\019\019_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\019\019_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\019\019_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\019\019_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\020\020_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\020\020_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\020\020_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\020\020_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\020\020_F_4.JPG
Processing

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\028\028_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\028\028_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\028\028_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\028\028_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\029\029_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\029\029_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\029\029_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\029\029_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\029\029_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\029\029_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\029\029_F_6.JPG
Processing

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\037\037_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\037\037_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\038\038_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\038\038_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\038\038_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\038\038_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\038\038_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\038\038_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\038\038_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\038\038_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\038\038_F_8.JPG
Processing

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\047\047_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\047\047_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\047\047_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\047\047_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\047\047_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\047\047_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\047\047_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\047\047_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\047\047_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\047\047_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\048\048_F_0.JPG
Processing

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\057\057_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\057\057_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\057\057_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\057\057_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\057\057_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\057\057_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\057\057_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\057\057_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\058\058_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\058\058_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\058\058_F_2.JPG
Processing

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\066\066_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\066\066_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\066\066_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\066\066_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\066\066_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\066\066_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\067\067_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\067\067_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\067\067_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\067\067_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\067\067_F_4.JPG
Processing

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\075\075_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\075\075_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\075\075_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\075\075_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\076\076_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\076\076_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\076\076_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\076\076_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\076\076_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\076\076_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\076\076_F_6.JPG
Processing

Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\084\084_F_8.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\084\084_F_9.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\085\085_F_0.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\085\085_F_1.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\085\085_F_2.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\085\085_F_3.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\085\085_F_4.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\085\085_F_5.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\085\085_F_6.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\085\085_F_7.JPG
Processing file C:/Users/Vineet Roy/Downloads/Dataset/palmprint/processed\085\085_F_8.JPG
Processing

In [43]:
import os
import random
import shutil

In [44]:
input_dir = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/augmented'

In [45]:
# Set up directories
train_dir1 = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/trainTestVal/train/1'
val_dir1 = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/trainTestVal/validation/1'
test_dir1 = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/trainTestVal/test/1'

train_dir0 = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/trainTestVal/train/0'
val_dir0 = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/trainTestVal/validation/0'
test_dir0 = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/trainTestVal/test/0'

train_dir_t = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/trainTestVal/train/t'
val_dir_t = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/trainTestVal/validation/t'
test_dir_t = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/trainTestVal/test/t'

In [46]:
num_train = 30
num_val = 6
num_test = 3

In [47]:
if not os.path.exists(train_dir1):
    os.makedirs(train_dir1)
if not os.path.exists(val_dir1):
    os.makedirs(val_dir1)
if not os.path.exists(test_dir1):
    os.makedirs(test_dir1)
if not os.path.exists(train_dir0):
    os.makedirs(train_dir0)
if not os.path.exists(val_dir0):
    os.makedirs(val_dir0)
if not os.path.exists(test_dir0):
    os.makedirs(test_dir0)
if not os.path.exists(train_dir_t):
    os.makedirs(train_dir_t)
if not os.path.exists(val_dir_t):
    os.makedirs(val_dir_t)
if not os.path.exists(test_dir_t):
    os.makedirs(test_dir_t)

In [48]:
train_file_index = 0
val_file_index = 0
test_file_index = 0

In [49]:
# Loop over classes
for class_folder in os.listdir(input_dir):
    print('processing folder: '+class_folder)
    class_path = os.path.join(input_dir, class_folder)
    if os.path.isdir(class_path):
        # Get list of image files
        img_files = os.listdir(class_path)
        template_file = [f for f in img_files if f.lower().endswith('00.jpg')][0]
        # Remove template image
        img_files.remove(template_file)
        # Randomly select 30 images for training
        train_imgs_p = random.sample(img_files, num_train)
        # Randomly select 6 images for validation
        val_imgs_p = random.sample(list(set(img_files) - set(train_imgs_p)), num_val)
        # Use remaining 3 images for testing
        test_imgs_p = list(set(img_files) - set(train_imgs_p) - set(val_imgs_p))

        other_classes = [c for c in os.listdir(input_dir) if c != class_folder]
        img_files = []
        for folder in other_classes:
            img_list = os.listdir(os.path.join(input_dir, folder))
            img_list_w_path = [os.path.join(input_dir,folder, img) for img in img_list]
            img_files.extend(img_list_w_path)

        # Randomly select 30 images for training
        train_imgs_n = random.sample(img_files, num_train)
        # Randomly select 6 images for validation
        val_imgs_n = random.sample(list(set(img_files) - set(train_imgs_n)), num_val)
        # Use remaining 3 images for testing
        test_imgs_n = random.sample(list(set(img_files) - set(train_imgs_n) - set(val_imgs_n)), num_test)

        # Create pairs for training
        for img_file_p, img_file_n in zip(train_imgs_p, train_imgs_n):
            # Create a unique index for each pair
            train_file_index += 1
            copiedName = f'img{train_file_index:04d}.jpg'
            # Copy template file
            shutil.copy(os.path.join(class_path, template_file), os.path.join(train_dir_t, copiedName))
            # Copy positive sample
            shutil.copy(os.path.join(class_path, img_file_p), os.path.join(train_dir1, copiedName))
            # Copy negative sample
            shutil.copy(os.path.join(class_path, img_file_n), os.path.join(train_dir0, copiedName))

        # Create pairs for validation
        for img_file_p, img_file_n in zip(val_imgs_p, val_imgs_n):
            # Create a unique index for each pair
            val_file_index += 1
            copiedName = f'img{val_file_index:04d}.jpg'
            # Copy template file
            shutil.copy(os.path.join(class_path, template_file), os.path.join(val_dir_t, copiedName))
            # Copy positive sample
            shutil.copy(os.path.join(class_path, img_file_p), os.path.join(val_dir1, copiedName))
            # Copy negative sample
            shutil.copy(os.path.join(class_path, img_file_n), os.path.join(val_dir0, copiedName))

        # Create pairs for testing
        for img_file_p, img_file_n in zip(test_imgs_p, test_imgs_n):
            # Create a unique index for each pair
            test_file_index += 1
            copiedName = f'img{test_file_index:04d}.jpg'
            # Copy template file
            shutil.copy(os.path.join(class_path, template_file), os.path.join(test_dir_t, copiedName))
            # Copy positive sample
            shutil.copy(os.path.join(class_path, img_file_p), os.path.join(test_dir1, copiedName))
            # Copy negative sample
            shutil.copy(os.path.join(class_path, img_file_n), os.path.join(test_dir0, copiedName))

processing folder: 001
processing folder: 002
processing folder: 003
processing folder: 004
processing folder: 005
processing folder: 006
processing folder: 007
processing folder: 008
processing folder: 009
processing folder: 010
processing folder: 011
processing folder: 012
processing folder: 013
processing folder: 014
processing folder: 015
processing folder: 016
processing folder: 017
processing folder: 018
processing folder: 019
processing folder: 020
processing folder: 021
processing folder: 022
processing folder: 023
processing folder: 024
processing folder: 025
processing folder: 026
processing folder: 027
processing folder: 028
processing folder: 029
processing folder: 030
processing folder: 031
processing folder: 032
processing folder: 033
processing folder: 034
processing folder: 035
processing folder: 036
processing folder: 037
processing folder: 038
processing folder: 039
processing folder: 040
processing folder: 041
processing folder: 042
processing folder: 043
processing 

In [50]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

In [51]:
train_dir1 = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/trainTestVal/train/1'
img_files = os.listdir(train_dir1)

In [52]:
r_sum = g_sum = b_sum = 0
for file in tqdm(img_files):
    img = cv2.imread(os.path.join(train_dir1, file))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    r_sum += np.sum(img[:, :, 0])/255
    g_sum += np.sum(img[:, :, 1])/255
    b_sum += np.sum(img[:, :, 2])/255

100%|█████████████████████████████████████████████████████████████████████████████| 2760/2760 [00:26<00:00, 103.21it/s]


In [53]:
[img_sz_y, img_sz_x, _] = img.shape
r_avg = r_sum/(len(img_files)*img_sz_y*img_sz_x)
g_avg = g_sum/(len(img_files)*img_sz_y*img_sz_x)
b_avg = b_sum/(len(img_files)*img_sz_y*img_sz_x)

In [54]:
rgb_dict = { 'r': [r_avg], 'g': [g_avg], 'b':[b_avg]}

In [55]:
df = pd.DataFrame(rgb_dict)
df.to_csv('mean_rgb_val.csv', index=False)

In [56]:
df

,r,g,b
0,0.6412,0.522644,0.516483


In [57]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, Input, Lambda, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import Input
from tensorflow.keras.models import load_model
from keras.callbacks import ModelCheckpoint

In [58]:
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools
import os
import random
import numpy as np

In [59]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print('Num GPUs Available: ', len(physical_devices))
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num GPUs Available:  0


In [60]:
M = 408
N = 306

In [61]:
batch_sz = 16
num_epochs = 48

In [62]:
train_path = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/trainTestVal/train'
val_path = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/trainTestVal/validation'
test_path = 'C:/Users/Vineet Roy/Downloads/Dataset/palmprint/trainTestVal/test'
classNames = ['no_match', 'match']

In [63]:
# generates pairs of template + matching image and template + non-matching image
def generate_pairs(directory, shuffle=True):
    template_folder = os.path.join(directory, 't')
    pos_folder = os.path.join(directory, '1')
    neg_folder = os.path.join(directory, '0')

    num_images = len(os.listdir(template_folder))

    # Generate positive pairs
    pos_pairs = []
    template_idx = 0
    for i in range(num_images):
        template_idx += 1
        pos_idx = template_idx  # Use the same index for template and positive image
        template_file = os.path.join(template_folder, 'img{:04d}.jpg'.format(template_idx))
        pos_file = os.path.join(pos_folder, 'img{:04d}.jpg'.format(pos_idx))
        pos_pairs.append([template_file, pos_file, 1])
    # Generate negative pairs
    neg_pairs = []
    template_idx = 0
    for i in range(num_images):
        template_idx += 1
        neg_idx = template_idx  # Use the same index for template and neg image
        template_file = os.path.join(template_folder, 'img{:04d}.jpg'.format(template_idx))
        neg_file = os.path.join(neg_folder, 'img{:04d}.jpg'.format(neg_idx))
        neg_pairs.append([template_file, neg_file, 0])

    all_pairs = pos_pairs + neg_pairs
    if shuffle:
        random.shuffle(all_pairs)

    return all_pairs

In [64]:
# splits images and target value
def separateList(dataset_list):
    dataset_pairs = [[lst[0], lst[1]] for lst in dataset_list]
    dataset_labels = [lst[2] for lst in dataset_list]
    return dataset_pairs, dataset_labels

In [65]:
# function to load and preprocess images
def load_and_preprocess_image(image_path, mean_rgb):
    # load image from file path
    image = tf.io.read_file(image_path)
    # decode jpeg encoded image
    image = tf.image.decode_jpeg(image, channels=3)
    # normalize pixel values to be in the range [0, 1] and subtract r,g,b mean
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.subtract(image, mean_rgb)
    return image


In [66]:
# confusion matrix generation
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalizes Confusion Matrix")
    else:
        print("Confusion Matrix, without normalization")
    print(cm)
    thresh = cm.max()/2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [67]:
#get mean RGB for preprocessing
mean_rgb_file = 'mean_rgb_val.csv'
if os.path.isfile(mean_rgb_file):
    mean_rgb = pd.read_csv(mean_rgb_file)
    mean_rgb = np.array(mean_rgb)
else:
    print(f'Error: CSV file "{mean_rgb_file}" not found.')
    mean_rgb = np.array([0.641855879, 0.523251229, 0.51696453])  # load default

In [68]:
# generate image pairs and labels for train, val and test. Test and Val sets are shuffled
train_set = generate_pairs(train_path)
val_set = generate_pairs(val_path)
test_set = generate_pairs(test_path, False)

train_pairs, train_labels = separateList(train_set)
val_pairs, val_labels = separateList(val_set)
test_pairs, test_labels = separateList(test_set)

train_labels = np.array(train_labels)
val_labels = np.array(val_labels)
test_labels = np.array(test_labels)

In [69]:
# create training dataset from pairs and labels
train_dataset = tf.data.Dataset.from_tensor_slices((train_pairs, train_labels))
#map image loading and preprocessing function to the training pairs
train_dataset = train_dataset.map(lambda x, y: ((load_and_preprocess_image(x[0], mean_rgb), load_and_preprocess_image(x[1], mean_rgb)), y))
# batch the training dataset
train_dataset = train_dataset.batch(batch_sz)

In [70]:
# create validation dataset from pairs and labels
val_dataset = tf.data.Dataset.from_tensor_slices((val_pairs, val_labels))
# map image loading and preprocessing function to the validation pairs
val_dataset = val_dataset.map(lambda x, y: ((load_and_preprocess_image(x[0], mean_rgb), load_and_preprocess_image(x[1], mean_rgb)), y))
# batch the validation dataset
val_dataset = val_dataset.batch(batch_sz)

In [71]:
# create validation dataset from pairs and labels
test_dataset = tf.data.Dataset.from_tensor_slices((test_pairs, test_labels))
# map image loading and preprocessing function to the validation pairs
test_dataset = test_dataset.map(lambda x, y: ((load_and_preprocess_image(x[0], mean_rgb), load_and_preprocess_image(x[1], mean_rgb)), y))
test_dataset = test_dataset.batch(16)

In [72]:
input_shape = (M, N, 3)
template_input = Input(input_shape)
test_input = Input(input_shape)

In [73]:
# left and right twin branches of the network
convnet = Sequential([
    Conv2D(8, (23, 23), activation='relu', input_shape=input_shape),
    BatchNormalization(),
    Activation('relu'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(16, (13, 13), activation='relu'),
    BatchNormalization(),
    Activation('relu'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(32, (7, 7), activation='relu'),
    BatchNormalization(),
    Activation('relu'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(48, (5, 5), activation='relu'),
    BatchNormalization(),
    Activation('relu'),
    MaxPool2D(pool_size=(2, 2), strides=2),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    Activation('relu'),
    MaxPool2D(pool_size=(2, 2), strides=1),
    Conv2D(96, (3, 3), activation='relu'),
    BatchNormalization(),
    Activation('relu'),
    MaxPool2D(pool_size=(2, 2), strides=1),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    Activation('relu'),
    Flatten(),
])

In [74]:
print("convnet summary:")
convnet.summary()

convnet summary:
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 386, 284, 8)       12704     
                                                                 
 batch_normalization (BatchN  (None, 386, 284, 8)      32        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 386, 284, 8)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 193, 142, 8)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 181, 130, 16)      21648     
                                                                 
 batch_normalization_1 (Batc  (None, 18

In [75]:
encoded_l = convnet(template_input)
encoded_r = convnet(test_input)

In [76]:
# merged part of the network
merged = concatenate([encoded_l, encoded_r])
merged = Dense(512, activation='relu')(merged)
merged = Dropout(0.15)(merged)
merged = Dense(128, activation='relu')(merged)
merged = Dropout(0.15)(merged)
prediction = Dense(1, activation='sigmoid')(merged)

siamese_model = Model(inputs=[template_input, test_input], outputs=prediction)

siamese_model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.005), metrics=['accuracy'])

print("siamese summary:")
siamese_model.summary()

siamese summary:
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 408, 306, 3  0           []                               
                                )]                                                                
                                                                                                  
 input_2 (InputLayer)           [(None, 408, 306, 3  0           []                               
                                )]                                                                
                                                                                                  
 sequential (Sequential)        (None, 5632)         293312      ['input_1[0][0]',                
                                                                  'input_2[0]

In [77]:
# set checkpoints to save after each epoch
checkpoint_filepath = './models/model_checkpoint.h5'
os.makedirs('./models', exist_ok=True)

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [78]:
# continue training from last checkpoint if the model was trained earlier
if os.path.isfile('./models/model_checkpoint.h5'):
    siamese_model = load_model('./models/model_checkpoint.h5')

In [ ]:
siamese_model.fit(x=train_dataset, validation_data=val_dataset, batch_size=batch_sz, epochs=d, callbacks=[model_checkpoint_callback])


Epoch 1/48
345/345 [==============================] - 6342s 18s/step - loss: 0.9669 - accuracy: 0.6342 - val_loss: 0.5804 - val_accuracy: 0.7047
Epoch 2/48
254/345 [=====================>........] - ETA: 27:14 - loss: 0.5592 - accuracy: 0.7306

In [ ]:
predictions = siamese_model.predict(x=test_dataset)
rounded_predictions = np.floor(predictions + 0.5).astype(int)

In [ ]:
cm = confusion_matrix(y_true=test_labels, y_pred=rounded_predictions)
cm_plot_labels = ['no_match', 'match']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

In [ ]:
print("Test Accuracy = ")
print(np.sum(test_labels == rounded_predictions.reshape(1, len(rounded_predictions)))/len(test_labels))